In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import pca_grid_search

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import itertools

#variables para guardar los resultados
tipo = pruebas[3]
dataset_name = datasets[0]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Adaptación

## Creación de modelos de adaptación.

Por cada par de dominios se entrenan distintos modelos según los parámetros establecidos.
Al entrenar cada modelo, este es inmediatamente probado, siendo almacenado o descartado según su desempeño.

Cada modelo es guardado en la ruta: models/amazon/pca/me2\_[dimensiones]\_[dominio\_fuente]_[dominio\_objetivo].pkl

# Pruebas con el dataset de Amazon (3000 Dimensiones)

In [3]:
dims = 3000

In [4]:
print tipo
print dataset_name
print dims
print data_path

pca
amazon
3000
data


In [5]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [6]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me2_%d_models_paths.pkl' % dims)

parameters = {
    'n_components': [int(dims/2), int(dims/3), int(dims/4)],
}

pairs = list(itertools.permutations(domains, 2))
i = 1

#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega
print "Adaptando entre:"
for src, tgt in pairs:
    print "%d) %s - %s" % (i, src, tgt)
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()

    X_train = np.concatenate((X_src, X_tgt))

    best_model, score = pca_grid_search(X_train, X_src, y_src, parameters, n_jobs=4)
    print "\tMejor modelo: %.3f" % score

    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)

    best_model_save_path = os.path.join(folder_path, prefix)
    print "\tGuardando modelo en %s\n" % best_model_save_path
    joblib.dump(best_model, best_model_save_path)
    i = i+1

print "\nAdaptaciones completadas"

Adaptando entre:
1) electronics - dvd
	n_components: 1500
	0.890

	n_components: 1000
	0.884

	n_components: 750
	0.876

	Mejor modelo: 0.890
	Guardando modelo en models/amazon/pca/me2_3000_electronics_dvd.pkl

2) electronics - kitchen
	n_components: 1500
	0.886

	n_components: 1000
	0.880

	n_components: 750
	0.875

	Mejor modelo: 0.886
	Guardando modelo en models/amazon/pca/me2_3000_electronics_kitchen.pkl

3) electronics - books
	n_components: 1500
	0.890

	n_components: 1000
	0.881

	n_components: 750
	0.874

	Mejor modelo: 0.890
	Guardando modelo en models/amazon/pca/me2_3000_electronics_books.pkl

4) dvd - electronics
	n_components: 1500
	0.824

	n_components: 1000
	0.805

	n_components: 750
	0.813

	Mejor modelo: 0.824
	Guardando modelo en models/amazon/pca/me2_3000_dvd_electronics.pkl

5) dvd - kitchen
	n_components: 1500
	0.825

	n_components: 1000
	0.803

	n_components: 750
	0.804

	Mejor modelo: 0.825
	Guardando modelo en models/amazon/pca/me2_3000_dvd_kitchen.pkl

6) dvd - 

## Pruebas de clasificación

In [7]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
            
    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### PCA ####
    #############
    # se adaptan los dominios usando PCA
    print "Adaptando dominios..."
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)

    best_model_save_path = os.path.join(folder_path, prefix)

    best_pca = joblib.load(best_model_save_path)


    X_tr = labeled[src]['X_tr'][:, :dims].todense()
    X_tr_a = best_pca.transform(X_tr)
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    X_ts_a = best_pca.transform(X_ts)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-clf.score(X_ts_a, y_ts)


    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['PCA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i += 1
            
print "\nPruebas completadas."

Tarea 1 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 7 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 8 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 9 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 10 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 11 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 12 de 12
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas.


In [8]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,PCA,e->d,electronics,dvd,15.762894,27.385685,11.622791
1,PCA,e->k,electronics,kitchen,8.587715,10.370259,1.782545
2,PCA,e->b,electronics,books,15.157879,29.430736,14.272857
3,PCA,d->e,dvd,electronics,11.127778,25.290632,14.162854
4,PCA,d->k,dvd,kitchen,8.587715,20.175504,11.587790
5,PCA,d->b,dvd,books,15.157879,26.375659,11.217780
6,PCA,k->e,kitchen,electronics,11.127778,12.937823,1.810045
7,PCA,k->d,kitchen,dvd,15.762894,27.105678,11.342784
8,PCA,k->b,kitchen,books,15.157879,30.208255,15.050376
9,PCA,b->e,books,electronics,11.127778,20.245506,9.117728


In [9]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/pca/me2_3000.csv
Resultados guardados.


# Pruebas con el dataset de Amazon (1000 Dimensiones)

In [10]:
dims = 1000

In [11]:
print tipo
print dataset_name
print dims
print data_path

pca
amazon
1000
data


In [12]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [13]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me2_%d_models_paths.pkl' % dims)

parameters = {
    'n_components': [int(dims/2), int(dims/3), int(dims/4)],
}

pairs = list(itertools.permutations(domains, 2))
i = 1

#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega
print "Adaptando entre:"
for src, tgt in pairs:
    print "%d) %s - %s" % (i, src, tgt)
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()

    X_train = np.concatenate((X_src, X_tgt))

    best_model, score = pca_grid_search(X_train, X_src, y_src, parameters, n_jobs=4)
    print "\tMejor modelo: %.3f" % score

    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)

    best_model_save_path = os.path.join(folder_path, prefix)
    print "\tGuardando modelo en %s\n" % best_model_save_path
    joblib.dump(best_model, best_model_save_path)
    i = i+1

print "\nAdaptaciones completadas"

Adaptando entre:
1) electronics - dvd
	n_components: 500


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	0.828

	n_components: 333


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	0.842

	n_components: 250


...

	0.833

	Mejor modelo: 0.833
	Guardando modelo en models/amazon/pca/me2_1000_electronics_books.pkl

4) dvd - electronics
	n_components: 500


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	0.794

	n_components: 333


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	0.796

	n_components: 250
	0.795

	Mejor modelo: 0.796
	Guardando modelo en models/amazon/pca/me2_1000_dvd_electronics.pkl

5) dvd - kitchen
	n_components: 500
	0.772

	n_components: 333
	0.790

	n_components: 250
	0.794

	Mejor modelo: 0.794
	Guardando modelo en models/amazon/pca/me2_1000_dvd_kitchen.pkl

6) dvd - books
	n_components: 500
	0.784

	n_components: 333
	0.777

	n_components: 250
	0.801

	Mejor modelo: 0.801
	Guardando modelo en models/amazon/pca/me2_1000_dvd_books.pkl

7) kitchen - electronics
	n_components: 500
	0.846

	n_components: 333
	0.853

	n_components: 250
	0.848

	Mejor modelo: 0.853
	Guardando modelo en models/amazon/pca/me2_1000_kitchen_electronics.pkl

8) kitchen - dvd
	n_components: 500
	0.846

	n_components: 333
	0.848

	n_components: 250
	0.856

	Mejor modelo: 0.856
	Guardando modelo en models/amazon/pca/me2_1000_kitchen_dvd.pkl

9) kitchen - books
	n_components: 500
	0.840

	n_components: 333
	0.856

	n_components: 250
	0.857

	Mejor modelo: 0.857
	Guard

## Pruebas de clasificación

In [14]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
            
    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### PCA ####
    #############
    # se adaptan los dominios usando PCA
    print "Adaptando dominios..."
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)

    best_model_save_path = os.path.join(folder_path, prefix)

    best_pca = joblib.load(best_model_save_path)


    X_tr = labeled[src]['X_tr'][:, :dims].todense()
    X_tr_a = best_pca.transform(X_tr)
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    X_ts_a = best_pca.transform(X_ts)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-clf.score(X_ts_a, y_ts)


    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['PCA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i += 1
            
print "\nPruebas completadas."

Tarea 1 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 7 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 8 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 9 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 10 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 11 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 12 de 12
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas.


In [15]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,PCA,e->d,electronics,dvd,16.900423,26.840671,9.940249
1,PCA,e->k,electronics,kitchen,16.215405,14.762869,-1.452536
2,PCA,e->b,electronics,books,23.710593,31.910798,8.200205
3,PCA,d->e,dvd,electronics,16.397910,28.853221,12.455311
4,PCA,d->k,dvd,kitchen,16.215405,24.558114,8.342709
5,PCA,d->b,dvd,books,23.710593,23.230581,-0.480012
6,PCA,k->e,kitchen,electronics,16.397910,17.910448,1.512538
7,PCA,k->d,kitchen,dvd,16.900423,27.788195,10.887772
8,PCA,k->b,kitchen,books,23.710593,32.783320,9.072727
9,PCA,b->e,books,electronics,16.397910,26.215655,9.817745


In [16]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/pca/me2_1000.csv
Resultados guardados.
